In [ ]:
import keras
from keras.models import Sequential
from keras.layers import MaxPooling2D, Dense, Dropout, Flatten, Conv2D, BatchNormalization, Activation
import os

In [ ]:
batch_size = 32
num_classes = 10
epochs = 100
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [ ]:
# Loading Dataset (CIFAR10: Subset of CIFAR)
from keras.datasets import cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
#Note that I'll be using Batch Normalization instead of LRN
def create_alexnet():
    inputShape = (227,227,3)   # Note that input shape is mentioned as 224x224x3 in AlexNet Paper. 
                               #It actually is 227X227X3 as mentioned by Andrej Karpathy in CS231N
    
    alexnet = Sequential()
    
    # Layer 1.
        # [Input] ==> 227x227x3
        # --> 227x227x3 ==> [Convolution: size=(11x11x3)x96, strides=4, padding=valid] ==> 55x55x96
        # --> 55x55x96 ==> [ReLU] ==> 55x55x96
        # --> 55x55x96 ==> [Batch Normalization] ==> 55x55x96
        # --> 55x55x96 ==> [Max-Pool: size=3x3, strides=2, padding=valid] ==> 27x27x96
        # --> [Output] ==> 27x27x96
    alexnet.add(Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), input_shape = (227,227,3), padding = 'valid'))
    alexnet.add(Activation('relu'))
    alexnet.add(BatchNormalization(axis = 1))
    alexnet.add(MaxPooling2D(pool_size = (3,3), strides = 2, padding = 'valid'))
    
    # Layer 2.
        # [Input] ==> 27x27x96
        # --> 27x27x96 ==> [Convolution: size=(5x5x96)x256, strides=1, padding=same] ==> 27x27x256
        # --> 27x27x256 ==> [ReLU] ==> 27x27x256
        # --> 27x27x256 ==> [Batch Normalization] ==> 27x27x256
        # --> 27x27x256 ==> [Max-Pool: size=3x3, strides=2, padding=valid] ==> 13x13x256
        # --> [Output] ==> 13x13x256
    alexnet.add(Conv2D(filters = 256, kernel_size = (5,5), strides = (1,1), padding= 'same'))
    alexnet.add(Activation('relu'))
    alexnet.add(BatchNormalization(axis = 1))
    alexnet.add(MaxPooling2D(pool_size = (3,3), strides = 2, padding = 'valid'))
    
    # Layer 3.
        # [Input] ==> 13x13x256
        # --> 13x13x256 ==> [Convolution: size=(3x3x256)x384, strides=1, padding=same] ==> 13x13x384
        # --> 13x13x384 ==> [ReLU] ==> 13x13x384
        # --> [Output] ==> 13x13x384
    alexnet.add(Conv2D(filters = 384, kernel_size = (3,3), strides = (1,1), padding='same'))
    alexnet.add(Activation('relu'))
    
    # Layer 4.
        # [Input] ==> 13x13x384
        # --> 13x13x384 ==> [Convolution: size=(3x3x384)x384, strides=1, padding=same] ==> 13x13x384
        # --> 13x13x384 ==> [ReLU] ==> 13x13x384
        # --> [Output] ==> 13x13x384
    alexnet.add(Conv2D(filters = 384, kernel_size = (3,3), strides = (1,1), padding='same'))
    alexnet.add(Activation('relu'))
    
    # Layer 5.
        # [Input] ==> 13x13x384
        # --> 13x13x384 ==> [Convolution: size=(3x3x384)x256, strides=1, padding=same] ==> 13x13x256
        # --> 13x13x256 ==> [ReLU] ==> 13x13x256
        # --> 13x13x256 ==> [Max-Pool: size=3x3, strides=2, padding=valid] ==> 6x6x256
        # --> [Output] ==> 6x6x256
    alexnet.add(Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size = (3,3), strides = 2, padding = 'valid'))
    
    # Layer 6.
        # [Input] ==> 6x6x256=9216
        # --> 9216 ==> [Fully Connected: neurons=4096] ==> 4096
        # --> 4096 ==> [ReLU] ==> 4096
        # --> 4096 ==> [Dropout] ==> 4096
        # --> [Output] ==> 4096
    alexnet.add(Flatten())
    alexnet.add(Dense(4096))
    alexnet.add(Activation("relu"))
    alexnet.add(Dropout(0.4))
    
    # Layer 7.
        # [Input] ==> 4096
        # --> 4096 ==> [Fully Connected: neurons=4096] ==> 4096
        # --> 4096 ==> [ReLU] ==> 4096
        # --> 4096 ==> [Dropout] ==> 4096
        # --> [Output] ==> 4096
    
    alexnet.add(Dense(4096))
    alexnet.add(Activation("relu"))
    alexnet.add(Dropout(0.4))
    
    # Layer 8.
        # [Input] ==> 4096
        # --> 4096 ==> [Logits: neurons=1000] ==> 1000
        # --> [Output] ==> 1000
    alexnet.add(Dense(1000))
    alexnet.add(Activation("softmax"))
    
    alexnet.summary()
    return alexnet
    

In [ ]:
#Preprocessing data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = create_alexnet()

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
alexnet.compile(loss='mse',
              optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True,  # randomly flip images ;Mentioned in the paper)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])